# Rutinas para inferencia de institución y frecuencias de transmisión de datos de EMA, humedad relativa

> Elaborado por Paola Álvarez, profesional contratista IDEAM, contrato 196 de 2024. Comentarios o inquietudes, remitir a *palvarez@ideam.gov.co* 

In [3]:
import pandas as pd
import numpy as np
import os
import re

In [5]:
# Se muestra el caso con mayor cantidad de frecuencias encontradas según slicing de [-10:] | Se cambia a [-100:] 10/2023
def analizar_frecuencias(carpeta, chunk_size=50000):
    archivos = os.listdir(carpeta)
    resultados = []

    # Se recorre cada archivo en la carpeta
    for archivo in archivos:
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(carpeta, archivo)
        
            # Se procesan los archivos csv por fragmentos
            reader = pd.read_csv(ruta_archivo, encoding='latin-1', chunksize=chunk_size)
        
            periodo = []
            
            # Se itera sobre todos los chunk
            for chunk in reader:
                # De cada chunk se transforma a datetime la serie/columna 'Fecha'
                chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
                sttn_val = chunk['Station'].values[0]
                
                try:
                    # Obtener frecuencia
                    periodos = pd.infer_freq(chunk['Fecha'][-200:])
                    periodo.append(periodos)
                except ValueError as e:
                    print(f'Error al inferir la frecuencia en el archivo {archivo}: {str(e)}')
            
            # Agregar frecuencia al DataFrame
            if periodo:
                # Si se dedesa evaluar el comportamiento, se puede imprimir el siguiente statement
                #print(f'La frecuencia de los datos en el archivo {archivo} es {periodo[0]}')
                resultados.append({'Station': sttn_val, 'FreqInf': periodo[-1]})
    
    # Crear DataFrame con los resultados
    df_resultados = pd.DataFrame(resultados)
    return df_resultados

In [7]:
# Uso función 
carpeta = 'RawUnmodified_TS30'
df_resultados = analizar_frecuencias(carpeta) 
# Se copia en un bloc de notas y se guarda como un .csv

Error al inferir la frecuencia en el archivo Estacion_0024015110.csv: Need at least 3 dates to infer frequency


In [9]:
df_resultados.to_csv('freqs_TS30.csv')

Posteriormente, se hace una revisión manual de lo obtenido en el archivo 'frecinst100b.csv'. Se observan varias estaciones cuya frecuencia no pudo ser inferida que poseen comportamientos 'híbridos' a lo largo de sus transmisiones, por lo cual, se diligencia columna de observaciones, dejando en el registro la predominante.
Se exporta este nuevo archivo como 'frecinst100b_manual.csv' que es leído a continuación:

In [ ]:
## La siguiente, para comparar los dos resultados de [-10:] a [-100:] y 'FreqInf': periodo[0] a periodo[-1]
# Resultado [-100:]
freq100man = pd.read_csv('frecinst100b_manual.csv', sep=';')
# Resultado de inferencia de [-10:]:
freq10man = pd.read_csv('freq10binst_manualcorrect.csv')

In [ ]:
# Asumiendo que ambos DataFrames tienen una columna 'Station' en común
merged_df = pd.merge(freq100man, freq10man, on='Station')

# Crear una nueva columna 'diff' que sea True si 'FreqInf' es diferente de 'freq'
merged_df['diff'] = merged_df['FreqInf'] != merged_df['freq']

print(merged_df[merged_df['diff']])

# Filtrar las filas donde 'diff' es True
diff_stations = merged_df[merged_df['diff']]

# Seleccionar solo la columna 'Station' y obtener valores únicos
unique_stations = diff_stations['Station'].unique()

# Mostrar la lista de estaciones con diferencias
print(unique_stations.shape)

In [ ]:
merged_df['diff'] = (merged_df['FreqInf'] != merged_df['freq']) & pd.notna(merged_df['FreqInf'])

# Filtrar las filas donde 'diff' es True
diff_rows = merged_df[merged_df['diff']]

# Seleccionar solo las columnas 'FreqInf' y 'freq'
diff_values = diff_rows[['Station','FreqInf', 'freq','Observ']]

# Mostrar las diferencias
print(diff_values.shape)
print(diff_values)

In [15]:
CoordsTS30 = pd.read_excel('EMA_TS30cm_coord.xlsx')
CoordsTS30

,Unnamed: 0,Station,lat,long,entidad
0,0,11105020,7.154333,-76.977194,IDEAM
1,1,11135030,8.036722,-77.087861,IDEAM
2,2,12015100,7.940750,-76.696167,IDEAM
3,3,12015110,7.671139,-76.694056,IDEAM
4,4,13085050,9.252722,-75.844417,IDEAM
...,...,...,...,...,...
61,61,52055220,1.070708,-77.636936,IDEAM
62,62,53075020,2.311417,-77.258778,IDEAM
63,63,55015010,5.521083,-76.973167,IDEAM
64,64,57025020,2.962944,-78.174361,IDEAM


In [3]:
CoordsTS30.rename(columns={'CODIGO':'Station','LATITUD':'lat','LONGITUD':'long', 'ENTIDAD':'entidad'}, inplace=True)

In [19]:
freqs = pd.read_csv('freqs_TS30.csv')#, sep=';')
freqs

,Unnamed: 0,Station,FreqInf
0,0,11105020,NaN
1,1,11135030,h
2,2,12015100,NaN
3,3,12015110,h
4,4,13085050,NaN
...,...,...,...
60,60,52055220,h
61,61,53075020,h
62,62,55015010,NaN
63,63,57025020,NaN


In [21]:
partialdf_TS30 = pd.merge(CoordsTS10, freqs[['Station','FreqInf']], on='Station', how='left')
partialdf_TS30

,Unnamed: 0,Station,lat,long,entidad,FreqInf
0,0,11105020,7.154333,-76.977194,IDEAM,NaN
1,1,11135030,8.036722,-77.087861,IDEAM,h
2,2,12015100,7.940750,-76.696167,IDEAM,NaN
3,3,12015110,7.671139,-76.694056,IDEAM,h
4,4,13085050,9.252722,-75.844417,IDEAM,NaN
...,...,...,...,...,...,...
61,61,52055220,1.070708,-77.636936,IDEAM,h
62,62,53075020,2.311417,-77.258778,IDEAM,h
63,63,55015010,5.521083,-76.973167,IDEAM,NaN
64,64,57025020,2.962944,-78.174361,IDEAM,NaN


In [23]:
partialdf_TS30.to_csv('EMATS30_LatLonEntFreq.csv', index=False, encoding='latin-1')